In [ ]:
from dsc80_utils import *

# Lecture 10 - Web Scraping

## DSC 80, Fall 2023

## 📣 Announcements 📣

- Good job on the Midterm!
- Mid-quarter survey out, due **Thursday, Nov 9 at 11:59pm**.
    - https://forms.gle/khHDPRuhgTqZTW1e9
- Lab 6 released, due Monday.
- No discussion or OH on Friday (Veteran's Day).
- Project 3 released, due Friday, Nov 17.

## 📆 Agenda

- [ ] Review: Accessing HTML
- [ ] HTML basics
- [ ] Parsing HTML using BeautifulSoup
- [ ] Example: Scraping the HDSI faculty
- [ ] Example: Scraping quotes

## Review: Accessing HTML

**Goal**: Access information about HDSI faculty members from the HDSI Faculty page.

Let's start by making a `GET` request to the HDSI Faculty page and see what the resulting HTML looks like. 

In [ ]:
import requests
r = requests.get('https://datascience.ucsd.edu/faculty/')
r

Wow, that is gross looking! 😰 

- It is **raw** HTML, which web browsers use to display websites.
- The information we are looking for – faculty information – is in there somewhere, but we have to search for it and extract it, which we wouldn't have to do if we had an API.
- We'll now look at how HTML documents are structured and how to extract information from them.

### Best practices for scraping

1. **Send requests slowly** and be upfront about what you are doing!
2. Respect the policy published in the page's `robots.txt` file.
    - Many sites have a `robots.txt` file in their root directory, which contains a policy that allows or disallows automatic access to their site. 
3. Don't spoof your User-agent (i.e. don't try to trick the server into thinking you are a person).
4. Read the Terms of Service for the site and follow it.

### Consequences of irresponsible scraping

If you make too many requests:
* The server may block your IP Address.
* You may take down the website.
    - A journalist scraped and accidentally took down the Cook County Inmate Locater.
    - As a result, inmate's families weren't able to contact them while the site was down.

### Scraping vs. Using an API

- APIs: Made by organzation that hosts data
    - E.g. Twitter has an API (https://developer.twitter.com/en/docs/twitter-api)
    - Provides a code-friendly way to access data.
    - Get data as JSON (usually).
- Scraping: Your code emulates a web browser
    - Not supported by most organizations.
    - Get data as HTML.
    - Have to parse HTML to get the data you want.

## The anatomy of HTML documents

### What is HTML?

* HTML (HyperText Markup Language) is **the** basic building block of the internet. 
* It defines the content and layout of a webpage, and as such, it is what you get back when you scrape a webpage.
* See [this tutorial](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/HTML_basics) for more details.

For instance, here's the content of a very basic webpage.

In [ ]:
!cat data/lec10_ex1.html

Using `IPython.display.HTML`, we can render it directly in our notebook.

In [ ]:
from IPython.display import HTML
from pathlib import Path
HTML(filename=Path('data') / 'lec10_ex1.html')

### The anatomy of HTML documents

* **HTML document**: The totality of markup that makes up a webpage.

* **Document Object Model (DOM)**: The internal representation of a HTML document as a hierarchical **tree** structure.

* **HTML element**: An object in the DOM, such as a paragraph, header, or title.
* **HTML tags**: Markers that denote the **start** and **end** of an element, such as `<p>` and `</p>`.

<center><img src='imgs/dom.jpg'></center>

<center><a href='https://simplesnippets.tech/what-is-document-object-modeldom-how-js-interacts-with-dom/'>(source)</a></center>

### Useful tags to know


|Element|Description|
|:---|:---|
|`<html>`|the document|
|`<head>`|the header|
|`<body>`|the body|
|`<div>` |a logical division of the document|
|`<span>`|an *inline* logical division|
|`<p>`|a paragraph|
| `<a>`| an anchor (hyperlink)|
|`<h1>, <h2>, ...`| header(s) |
|`<img>`| an image |

There are many, many more. See [this article](https://en.wikipedia.org/wiki/HTML_element) for examples.

### Example: images and hyperlinks

Tags can have **attributes**, which further specify how to display information on a webpage.

For instance, `<img>` tags have `src` and `alt` attributes (among others):

```html
<img src="king-selfie.png" alt="A photograph of King Triton." width=500>
```

Hyperlinks have `href` attributes: 

```html
Click <a href="https://dsc80.com/project3">this link</a> to access Project 3.
```

What do you think this webpage looks like?

In [ ]:
!cat data/lec10_ex2.html

### The `<div>` tag

```html
<div style="background-color:lightblue">
  <h3>This is a heading</h3>
  <p>This is a paragraph.</p>
</div>
```

* The `<div>` tag defines a division or a "section" of an HTML document.
    * Think of a `<div>` as a "cell" in a Jupyter Notebook.

* The `<div>` element is often used as a container for other HTML elements to style them with CSS or to perform operations involving them using JavaScript.

* `<div>` elements often have attributes, **which are important when scraping**!

### Document trees

Under the document object model (DOM), HTML documents are trees. In DOM trees, child nodes are **ordered**.

<center>

<img src="imgs/webpage_anatomy.png" width="50%">

</center>    

What does the DOM tree look like for this document?

<center><img src="imgs/dom_tree.png" width="50%"></center>

## Parsing HTML using Beautiful Soup

### Beautiful Soup 🍜

* [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a Python HTML parser.
    - To "parse" means to "extract meaning from a sequence of symbols".
* **Warning:** Beautiful Soup 4 and Beautiful Soup 3 work differently, so make sure you are using and looking at documentation for Beautiful Soup 4.

### Example HTML document

To start, we'll work with the source code for an HTML page with the DOM tree shown below:

<center><img src="imgs/dom_tree_1.png" width="50%"></center>

The string `html_string` contains an HTML "document".

In [ ]:
html_string = '''
<html>
    <body>
      <div id="content">
        <h1>Heading here</h1>
        <p>My First paragraph</p>
        <p>My <em>second</em> paragraph</p>
        <hr>
      </div>
      <div id="nav">
        <ul>
          <li>item 1</li>
          <li>item 2</li>
          <li>item 3</li>
        </ul>
      </div>
    </body>
</html>
'''.strip()

In [ ]:
HTML(html_string)

### `BeautifulSoup` objects

`bs4.BeautifulSoup` takes in a string or file-like object representing HTML (`markup`) and returns a **parsed** document.

In [ ]:
import bs4

In [ ]:
bs4.BeautifulSoup?

Normally, we pass the result of a `GET` request to `bs4.BeautifulSoup`, but here we will pass our hand-crafted `html_string`.

In [ ]:
soup = ...
soup

`BeautifulSoup` objects have several useful attributes, e.g. `text`:

### Traversing through `descendants`

The `descendants` attribute traverses a `BeautifulSoup` tree using **depth-first traversal**.

Why depth-first? Elements closer to one another on a page are more likely to be related than elements further away.

<center><img src="imgs/dom_tree_1.png" width="60%"></center>

### Finding elements in a tree

Practically speaking, you will not use the `descendants` attribute (or the related `children` attribute) directly very often. Instead, you will use the following methods:

- `soup.find(tag)`, which finds the **first** instance of a tag (the first one on the page, i.e. the first one that DFS sees).
    - More general: `soup.find(name=None, attrs={}, recursive=True, text=None, **kwargs)`.
- `soup.find_all(tag)` will find **all** instances of a tag.


### Using `find`

Let's try and extract the first `<div>` subtree.

<center><img src="imgs/dom_tree_1.png" width="60%"></center>  

In [ ]:
soup

In [ ]:
div = ...
div

<center><img src="imgs/dom_subtree_1.png" width="30%"></center>  

Let's try and find the `<div>` element that has an `id` attribute equal to `'nav'`.

`find` will return the first occurrence of a tag, regardless of its depth in the tree.

### Using `find_all`

`find_all` returns a list of all matches.

### Node attributes
* The `text` attribute of a tag element gets the text between the opening and closing tags.
* The `attrs` attribute lists all attributes of a tag.
* The `get(key)` method gets the value of a tag attribute.

The `get` method must be called directly on the node that contains the attribute you're looking for.

In [ ]:
soup

## Example: Quote scraping

Consider the following webpage: https://quotes.toscrape.com/

<center><img src="imgs/quotes2scrape.png" width=60%></center>

- Goal: Extract quotes (and relevant metadata) into a dataframe.
- A good first step is to use the "inspect element" tool in our web browser.

Specifically, let's try to make a DataFrame that looks like the one below:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>quote</th>
      <th>author</th>
      <th>author_url</th>
      <th>tags</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</td>
      <td>Albert Einstein</td>
      <td>https://quotes.toscrape.com/author/Albert-Einstein</td>
      <td>change,deep-thoughts,thinking,world</td>
    </tr>
    <tr>
      <th>1</th>
      <td>“It is our choices, Harry, that show what we truly are, far more than our abilities.”</td>
      <td>J.K. Rowling</td>
      <td>https://quotes.toscrape.com/author/J-K-Rowling</td>
      <td>abilities,choices</td>
    </tr>
    <tr>
      <th>2</th>
      <td>“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</td>
      <td>Albert Einstein</td>
      <td>https://quotes.toscrape.com/author/Albert-Einstein</td>
      <td>inspirational,life,live,miracle,miracles</td>
    </tr>
  </tbody>
</table>

### The plan

Eventually, we will create a single function – `make_quote_df` – which takes in an integer `n` and returns a **DataFrame** with the quotes on the **first `n` pages** of https://quotes.toscrape.com/.

To do this, we will define several helper functions:

- `download_page(i)`, which downloads a **single page** (page `i`) and returns a `BeautifulSoup` object of the response.

- `process_quote(div)`, which takes in a `<div>` tree corresponding to a **single quote** and returns a `dict` containing all of the relevant information for that quote.

- `process_page(divs)`, which takes in a list of `<div>` trees corresponding to a **single page** and returns a DataFrame containing all of the relevant information for all quotes on that page.

Key principle: some of our helper functions will make **requests**, and others will **parse**, but none will do both! 
- Easier to debug and catch errors.
- Avoids unnecessary requests.

### Downloading a single page

In [ ]:
def download_page(i):
    url = f'https://quotes.toscrape.com/page/{i}'
    request = ...
    return ...

In `quote_df`, we will call `download_page` repeatedly – once for `i=1`, once for `i=2`, ..., `i = n`. For now, we will work with just page 1 (chosen arbitrarily).

In [ ]:
soup = ...

### Parsing a single page

Let's look at the page's source code (via "inspect element") to find where the quotes in the page are located.

In [ ]:
divs = ...

In [ ]:
divs[0]

From this `<div>`, we can extract the quote, author name, author's URL, and tags.

Let's implement our next function, `process_quote`, which takes in a `<div>` corresponding to a single quote and returns a **dict** containing the quote's information.

- Why use a `dict`? Passing `pd.DataFrame()` a list of `dict`s is an easy way to create a dataframe.

In [ ]:
def process_quote(div):
    quote = div.find('span', class_='text').text
    author = div.find('small', class_='author').text
    author_url = 'https://quotes.toscrape.com' + div.find('a').get('href')
    tags = div.find('meta', class_='keywords').get('content')
    
    return {'quote': quote, 'author': author, 'author_url': author_url, 'tags': tags}

In [ ]:
process_quote(...)

Our last helper function will take in a **list** of `<div>`s, call `process_quote` on each `<div>` in the list, and return a **DataFrame**.

In [ ]:
def process_page(divs):
    return ...

In [ ]:
process_page(divs)

### Putting it all together

In [ ]:
def make_quote_df(n):
    '''Returns a DataFrame containing the quotes on the first n pages of https://quotes.toscrape.com/.'''
    ...

In [ ]:
quotes = make_quote_df(...)
quotes.head()

The elements in the `'tags'` column are all strings, but they look like lists. This is not ideal, as we will see shortly.

## Example: Scraping the HDSI Faculty page

### Example

Let's try and extract a list of HDSI Faculty from https://datascience.ucsd.edu/faculty/.

- As usual, we start by opening our Developer Tools.
- As we can see, the HTML is much more complicated -- this is usually the case for websites in the wild.

In [ ]:
fac_response = requests.get('https://datascience.ucsd.edu/faculty/')
fac_response

In [ ]:
soup = bs4.BeautifulSoup(fac_response.text)

How do we get the `<div>`s that we want?

- Trial and error by adding / removing classes.

In [ ]:
divs = soup.find_all(
    'div',
    class_='...',
)

In [ ]:
len(divs)

Within here, we need to extract each faculty member's name. It seems like names are stored as text within the `<h4>` tag.

We can also extract job titles:

Let's create a DataFrame consisting of names and job titles for each faculty member.

Now we have a DataFrame!

What if we want to get faculty members' pictures? 

In [ ]:
from IPython.display import Image, display

display(Image(...))

## One-hot encoding

Let's take another look at the 'tags' column of the `quotes` dataframe:

In [ ]:
quotes.head(2)

What if we wanted to know: how many quotes are tagged as `inspirational`?

- We can **flatten** lists of tags so that there is **one column per unique tag**.
    - For example, consider the tag `'inspirational'`.
    - If a quote has a 1 in the `'inspirational'` column, it **was** tagged `'inspirational'`.
    - If a quote has a 0 in the `'inspirational'` column, it **was not** tagged `'inspirational'`.
- This process – of converting categorical variables into columns of 1s and 0s – is called **one-hot encoding**. We will revisit it in a few weeks.

### One-hot encoded tags

Let's combine this one-hot-encoded DataFrame with `df`.

In [ ]:
quotes_full = ...
quotes_full.head()

If we want all quotes tagged `'inspirational'`, we can simply query:

## Web Data in Practice

[The spread of true and false news online](https://www.science.org/doi/full/10.1126/science.aap9559) by Vosoughi et al. compared how true and false news spreads via Twitter:

> There is worldwide concern over false news and the possibility that it can influence political, economic, and social well-being. To understand how false news spreads, Vosoughi et al. used a data set of rumor cascades on Twitter from 2006 to 2017. About 126,000 rumors were spread by ∼3 million people. False news reached more people than the truth; the top 1% of false news cascades diffused to between 1000 and 100,000 people, whereas the truth rarely diffused to more than 1000 people. Falsehood also diffused faster than the truth. The degree of novelty and the emotional reactions of recipients may be responsible for the differences observed.

- Twitter API for Twitter data.
- Web scraping for fact-checking websites.

## Summary, next time

- Beautiful Soup is an HTML parser that allows us to (somewhat) easily extract information from HTML documents.
    - `soup.find` and `soup.find_all` are the functions you will use most often.
- When writing scraping code:
    - Use "inspect element" to identify the names of tags and attributes that are relevant to the information you want to extract.
    - Separate your logic for making requests and for parsing.

### Next time

Regular expressions!